In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import scipy.stats as stats
import scipy.io 

In [2]:
LO = 1.2

NO = 600

xO = np.linspace(0,LO,NO)
dxO = xO[1] - xO[0]

t_f = 9.6
t_N = 9600
t = np.linspace(0,t_f,t_N)
dt = t[1] - t[0]

W = 10


c1 = 0.5


re = 0.5
rw = 0.5

KO = np.zeros((W,t_N,NO))
RO = np.zeros((W,t_N,NO))


KO[0,0,:] = np.linspace(2, 1 ,NO)
RO[0,0,0] = (1/rw)*KO[0,0,0]
RO[0,0,NO-1] = re*KO[0,0,NO-1]
RO[0,0,:] = np.linspace(RO[0,0,0] , RO[0,0,NO-1]  ,NO)


sigma = 1 



In [3]:
dt/dxO

0.49921866861131364

In [4]:
for i in range(t_N-1):
    for w in range(0,W):
            if w == 0 :
                for j in range(1,NO):
                    KO[w,i+1,j] = KO[w,i,j] - (c1)*(dt/dxO)*((KO[w,i,j] - KO[w,i,j-1])) 
                    KO[w,i+1,0] = KO[w,i,0] - (c1)*(dt/dxO)*(KO[w,i,0] - rw*RO[w,i,0])  
                for j in range(0,NO-1):
                    RO[w,i+1,j] = RO[w,i,j] + (c1/3)*(dt/dxO)*((RO[w,i,j+1] - RO[w,i,j])) 
                    RO[w,i+1,NO-1] = RO[w,i,NO-1] + (c1/3)*(dt/dxO)*(re*KO[w,i,NO-1] - RO[w,i,NO-1]) 
            elif w == 1 :          
                for j in range(1,NO):
                    KO[w,i+1,j] = KO[w,i,j] - (c1)*(dt/dxO)*((KO[w,i,j] - KO[w,i,j-1])) + sigma*dt*(1/np.sqrt(t_f))
                    KO[w,i+1,0] = KO[w,i,0] - (c1)*(dt/dxO)*(KO[w,i,0] - rw*RO[w,i,0])  + sigma*dt*(1/np.sqrt(t_f))  
                for j in range(0,NO-1):
                    RO[w,i+1,j] = RO[w,i,j] + (c1/3)*(dt/dxO)*((RO[w,i,j+1] - RO[w,i,j])) + sigma*dt*(1/np.sqrt(t_f))
                    RO[w,i+1,NO-1] = RO[w,i,NO-1] + (c1/3)*(dt/dxO)*(re*KO[w,i,NO-1] - RO[w,i,NO-1]) + sigma*dt*(1/np.sqrt(t_f))
            else:
                for j in range(1,NO):
                    KO[w,i+1,j] = KO[w,i,j] - (c1)*(dt/dxO)*((KO[w,i,j] - KO[w,i,j-1])) + sigma*dt*np.sqrt(2/t_f)*np.cos(np.pi*(w-1)*t[i]/t_f)
                    KO[w,i+1,0] = KO[w,i,0] - (c1)*(dt/dxO)*(KO[w,i,0] - rw*RO[w,i,0])  + sigma*dt*np.sqrt(2/t_f)*np.cos(np.pi*(w-1)*t[i]/t_f)  
                for j in range(0,NO-1):
                    RO[w,i+1,j] = RO[w,i,j] + (c1/3)*(dt/dxO)*((RO[w,i,j+1] - RO[w,i,j])) + sigma*dt*np.sqrt(2/t_f)*np.cos(np.pi*(w-1)*t[i]/t_f)
                    RO[w,i+1,NO-1] = RO[w,i,NO-1] + (c1/3)*(dt/dxO)*(re*KO[w,i,NO-1] - RO[w,i,NO-1]) + sigma*dt*np.sqrt(2/t_f)*np.cos(np.pi*(w-1)*t[i]/t_f)

In [5]:
scipy.io.savemat('ToyProblem_Kelvin.mat', {'Kelvin': KO}) 
scipy.io.savemat('ToyProblem_Rossby.mat', {'Rossby': RO})

In [10]:
K_variance_W = np.zeros((t_N,NO))
R_variance_W = np.zeros((t_N,NO))


for i in range(t_N):
    for j in range(0,NO):    
        for w in range(1,W):
            K_variance_W[i,j] += KO[w,i,j]**2
            R_variance_W[i,j] += RO[w,i,j]**2  

In [11]:
scipy.io.savemat('Ocean_Problem_Kelvin_Variance.mat', {'Kelvin_variance': K_variance_W}) 
scipy.io.savemat('Ocean_Problem_Rossby_Variance.mat', {'Rossby_variance': R_variance_W})